# PyZX interoperability: phases and T-gates

In [ ]:
import os
import sys
import pyzx as zx
from pathlib import Path
from fractions import Fraction
from pyzx.utils import EdgeType

from topologiq.scripts.runner import runner
from topologiq.utils.interop_pyzx import pyzx_g_to_simple_g
from topologiq.utils.utils_zx_graphs import kind_to_zx_type

repository_root = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))
output_folder_path = f"{repository_root}/outputs/txt"
if repository_root not in sys.path:
    sys.path.insert(0, repository_root)

zx.settings.colors = zx.rgb_colors
%matplotlib widget

This notebook offers an example of how perform an algorithmic lattice surgery with ***topologiq*** in a way that permits transferring or applying the phases in the original PyZX graph to the lattice surgery produced by ***topologiq*** and using this information to determine which blocks in the final result are T-gates.

## Input: Preparing a PyZX graph

We'll use a graph that is large enough to allow a variety of spiders with and without phases but also sufficiently short to not have to wait too long for the result to be ready for manipulation.

Note that while we draw the graph using `zx.draw()`, we also save the matplotlib figure separately. We'll use it later.

In [ ]:
circuit_name = "pyzx_example_t_gates"

c = zx.Circuit(3)
c.add_gate("ZPhase", 0, phase=Fraction(1,2))
c.add_gate("CNOT", 0, 2)
c.add_gate("CNOT", 1, 2)
c.add_gate("CNOT", 0, 2)
c.add_gate("ZPhase", 1, phase=Fraction(1,1))
c.add_gate("ZPhase", 2, phase=Fraction(1,4))

pyzx_graph = c.to_graph()
pyzx_graph.add_edge(edge_pair=(8,5),  edgetype=EdgeType.SIMPLE)
pyzx_graph.add_to_phase(vertex=7, phase=Fraction(1/2))
pyzx_graph.add_to_phase(vertex=10, phase=Fraction(1/4))

zx.draw(pyzx_graph, labels=True)
fig_data = zx.draw_matplotlib(pyzx_graph, labels=True)

## Process: Running the algorithm

To feed the graph into the algorithm, we first need to convert the PyZX graph into ***topologiq***'s native format, a `simple_graph`: a simple dictionary of nodes and edges.

The conversion removes information that is not needed for foundational operations, i.e., converting the graph into a 3D version of itself.

We will recover that information later in the process.

In [ ]:
simple_graph = pyzx_g_to_simple_g(pyzx_graph)
print(simple_graph)

The next step is to give the `simple_graph` to ***topologiq*** and let it work its magic (it's not magic, but let's pretend it is).

In short, ***topologiq*** will traverse that graph using a Breadth-First Search (BFS) approach and exchange all spiders and edges in the original graph for a 3D version of themselves.

If/when the lattice surgery is complete, ***topologiq*** will produce:
- text results saved to `[root]/outputs/txt/` that includes the final surgery/space-time diagram,
- and related objects for programmatic use.

Additional visualisations options are available but come at a significant cost in runtimes. Check ***topologiq***'s [README](../../README.md) for details.

In [ ]:
# Define weights for the value function to to choose best of several valid paths per each edge based on: (length of path, number of beams broken by path)
# A negative value for length of path favours short paths.
# A negative value for number of beams broken by path favours placements that do not block potential open faces requiring connections.
VALUE_FUNCTION_HYPERPARAMS = (-1, -1)

# Define a desired length of beams
# The longer the beams, the more space between faces. This feature needs improvement but, in theory, increases the odds of success.
# Needs to be combined with an equal or larger `MAX_PATHFINDER_SEARCH_SPACE`
LENGTH_OF_BEAMS = 9

kwargs = {"weights": VALUE_FUNCTION_HYPERPARAMS, "length_of_beams": LENGTH_OF_BEAMS}

simple_graph_after_use, edge_pths, lattice_nodes, lattice_edges = runner(
    simple_graph,
    circuit_name,
    strip_boundaries=False,
    hide_boundaries=False,
    max_attempts=10,
    vis_options=("final", None),
    fig_data=fig_data,  # This is where we use the fig_data from PyZX, so it's overlaid on top of progress and final visualisations
    **kwargs
)

## Output: Transferring phases and identifying T-gates in outputs

It is helpful to visualise the relation between the original PyZX graph and the resulting lattice surgery / space-time diagram. This is possible by printing the edges of the original graph alongside their 3D correspondences.
- Short edges with three items (block-pipe-block) are edges cleared easily by simply placing a spider into the 3D space at a location determined optimal by the algorithm.
- Long edges containing more than three items are edges that had to be broken into segments, typically as a result of having been processed after its source and target nodes had already been placed in the 3D space and therefore had fixed coordinates and 3D kinds.

In [ ]:
zx.draw(pyzx_graph, labels=True)

if edge_pths:
    for key, edge in edge_pths.items():
        block_by_block = []
        nodes_and_edges = []
        for node in edge["pth_nodes"]:
            block_by_block.append(node[1])
            nodes_and_edges.append(kind_to_zx_type(node[1]))
        print(f"{key}: {'-'.join(block_by_block)} ({' - '.join(nodes_and_edges)})")

It is also helpful to print `lattice_nodes` and `lattice_edges`. They contain the resulting lattice surgery / space-time diagram in the form of a dictionary with nodes and edges.

Notice that some of the 3D blocks in `lattice_nodes` have the same IDs as the spiders in the original ZX graph. These blocks are a direct instantiation of the original ZX spider with equal ID. Blocks with IDs different to those in the original ZX graph are intermediary blocks added by the algorithm to clear edges in 3D.

In [ ]:
if lattice_nodes:
    for key, node in lattice_nodes.items():
        print(f"Node ID {key}: {node}.")

if lattice_edges:
    for key, edge in lattice_edges.items():
        print(f"Edge ID {key}: {edge[0]} (corresponding edge in ZX-graph: {edge[1]}).")

As ellaborated [here](https://arxiv.org/pdf/1902.03178) and [here](https://arxiv.org/pdf/1903.10477), non-Clifford phases are those which are not multiples of π/2. It is therefore possible to identify T-gates in the final lattice surgery / space-time diagram in two steps:
- Transfer the phases of the original spiders to their corresponding instance in `lattice_nodes`, 
- Use phases to to identify T-gates.

In [ ]:
map_of_phases = pyzx_graph.phases()
lattice_nodes_extended = {}

if lattice_nodes:
    for id, node in lattice_nodes.items():
        position, kind = node
        phase = map_of_phases[id] if id in map_of_phases.keys() else 0
        gate_type = "T" if (isinstance(phase, Fraction) and phase % Fraction(1/2) != Fraction(0/1)) else "Clifford"
        lattice_nodes_extended[id] = (position, kind, phase, gate_type)

Finally, we can review the result by printing it to screen and/or appending it to previously saved text outputs.

In [ ]:
for id, node in lattice_nodes_extended.items():
    print(f"{id}: {node}")

lines = []
if lattice_nodes_extended and lattice_edges:
    lines.append(
        "\n__________________________\nLATTICE SURGERY EXTENDED (Graph + Phases)\n"
    )
    for key, node in lattice_nodes_extended.items():
        lines.append(f"Node ID: {key}. Info: {node}\n")
    for key, edge_info in lattice_edges.items():
         lines.append(f"Edge ID: {key}. Kind: {edge_info[0]}. Original edge in ZX graph: {edge_info[1]} \n")

    Path(output_folder_path).mkdir(parents=True, exist_ok=True)
    with open(f"{output_folder_path}/{circuit_name}.txt", "a") as f:
        f.writelines(lines)
        f.close()

    print(f"\nResult saved to: <...>/{circuit_name}.txt")